## **Configuración Inicial del Notebook**

In [1]:
# Importar librerías básicas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Configuración visualizaciones
%matplotlib inline
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)

In [3]:
# Cargar el dataset
df = pd.read_csv("../data/raw/CVD_cleaned.csv")

# Mostrar primeras filas y estructura
df.head()
print("\nInfo del dataset:")
print(df.info())
print("\nEstadísticas descriptivas:")
df.describe(include="all")


Info del dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 308854 entries, 0 to 308853
Data columns (total 19 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   General_Health                308854 non-null  object 
 1   Checkup                       308854 non-null  object 
 2   Exercise                      308854 non-null  object 
 3   Heart_Disease                 308854 non-null  object 
 4   Skin_Cancer                   308854 non-null  object 
 5   Other_Cancer                  308854 non-null  object 
 6   Depression                    308854 non-null  object 
 7   Diabetes                      308854 non-null  object 
 8   Arthritis                     308854 non-null  object 
 9   Sex                           308854 non-null  object 
 10  Age_Category                  308854 non-null  object 
 11  Height_(cm)                   308854 non-null  float64
 12  Weight_(kg)              

,General_Health,Checkup,Exercise,Heart_Disease,Skin_Cancer,Other_Cancer,Depression,Diabetes,Arthritis,Sex,Age_Category,Height_(cm),Weight_(kg),BMI,Smoking_History,Alcohol_Consumption,Fruit_Consumption,Green_Vegetables_Consumption,FriedPotato_Consumption
count,308854,308854,308854,308854,308854,308854,308854,308854,308854,308854,308854,308854.000000,308854.000000,308854.000000,308854,308854.000000,308854.000000,308854.000000,308854.000000
unique,5,5,2,2,2,2,2,4,2,2,13,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN
top,Very Good,Within the past year,Yes,No,No,No,No,No,No,Female,65-69,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN
freq,110395,239371,239381,283883,278860,278976,246953,259141,207783,160196,33434,NaN,NaN,NaN,183590,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,170.615249,83.588655,28.626211,NaN,5.096366,29.835200,15.110441,6.296616
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.658026,21.343210,6.522323,NaN,8.199763,24.875735,14.926238,8.582954
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,91.000000,24.950000,12.020000,NaN,0.000000,0.000000,0.000000,0.000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,163.000000,68.040000,24.210000,NaN,0.000000,12.000000,4.000000,2.000000
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,170.000000,81.650000,27.440000,NaN,1.000000,30.000000,12.000000,4.000000
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,178.000000,95.250000,31.850000,NaN,6.000000,30.000000,20.000000,8.000000


### **Siguiente Paso: Análisis Detallado de Variables y Preprocesado Avanzado**

**Objetivo:**  
- Entender la relación entre las variables predictoras y la variable objetivo (`Heart_Disease`).  
- Preparar los datos para el modelado mediante técnicas de encoding, escalado y manejo de variables.
- **Justificación:**  
  - El dataset no tiene valores nulos, pero hay variables categóricas que necesitan encoding (One-Hot o Label).  
  - Las variables numéricas (`BMI`, `Height`, etc.) tienen escalas diferentes y deben normalizarse.  
  - La variable objetivo está desbalanceada (22% "Yes", 78% "No"), lo que puede afectar el modelo.  

El valor más frecuente (“top”) en la variable `Age_Category` es **65-69**, lo que indica que la mayor parte de la muestra está compuesta por personas adultas mayores, cerca de la tercera edad. A partir de los resultados de `df.describe(include="all")`, se pueden extraer otras conclusiones interesantes:

---

### Otras conclusiones posibles del análisis descriptivo:

1. **Predominio de buen estado de salud general declarado:**
   - El valor más frecuente en `General_Health` es **"Very Good"**. Esto sugiere que la mayoría de los participantes perciben su salud como muy buena.

2. **Frecuencia de chequeos médicos:**
   - En `Checkup`, la moda es **"Within the past year"**, lo que indica que la mayoría de las personas se han realizado un chequeo médico en el último año.

3. **Actividad física:**
   - En `Exercise`, la mayoría responde **"Yes"**, lo que sugiere que una gran parte de la muestra realiza actividad física.

4. **Prevalencia de enfermedades:**
   - Para variables como `Heart_Disease`, `Skin_Cancer`, `Other_Cancer`, `Depression`, `Diabetes`, y `Arthritis`, el valor más frecuente es **"No"**. Esto indica que, aunque la muestra es mayoritariamente adulta mayor, la mayoría no reporta haber tenido estas enfermedades.

5. **Distribución por sexo:**
   - En `Sex`, la moda es **"Female"**, por lo que hay más mujeres que hombres en la muestra.

6. **Consumo de alcohol, frutas y verduras:**
   - El consumo promedio de alcohol es bajo (media ≈ 5), y el de frutas y verduras es moderado (medias ≈ 30 y 15 respectivamente), pero hay una gran dispersión (desviaciones estándar altas), lo que indica mucha variabilidad en los hábitos alimenticios.

7. **Índice de Masa Corporal (BMI):**
   - El BMI promedio es de **28.6**, lo que corresponde a la categoría de **sobrepeso** según la OMS. Esto puede indicar un riesgo aumentado de enfermedades cardiovasculares en la muestra.

8. **Historial de tabaquismo:**
   - La mayoría de los participantes no tiene historial de tabaquismo (`Smoking_History` = "No").

9. **Desbalance en la variable objetivo:**
   - Solo el **22%** de la muestra tiene enfermedad cardíaca (`Heart_Disease` = "Yes"), mientras que el **78%** no la tiene. Esto es importante para el modelado, ya que indica un desbalance de clases.

---

### Resumen

- La muestra está compuesta principalmente por mujeres adultas mayores (65-69 años), que perciben su salud como muy buena, se realizan chequeos médicos anuales y no presentan enfermedades graves en su mayoría.
- El BMI promedio indica sobrepeso, lo que puede ser un factor de riesgo relevante.
- Hay una gran variabilidad en los hábitos de consumo de alcohol, frutas y verduras.
- La variable objetivo está desbalanceada, lo que debe considerarse en el análisis predictivo.


#### **1. Análisis de la Variable Objetivo**  

In [ ]:
# 1. Primero asegurémonos de que la variable objetivo está correctamente definida
# Asumiendo que 'Heart_Disease' es la columna objetivo y contiene valores 'Yes'/'No' o True/False

# Convertir a binario si es necesario (1 para enfermedad, 0 para no enfermedad)
df['Heart_Disease'] = df['Heart_Disease'].map({'Yes': 1, 'No': 0})  # Ajusta según tus datos

# 2. Análisis de distribución
print("\nDistribución de Heart_Disease:")
distribucion = df['Heart_Disease'].value_counts(normalize=True) * 100
print(distribucion)

# 3. Visualización
plt.figure(figsize=(8, 5))
sns.countplot(x='Heart_Disease', data=df)
plt.title('Distribución de Enfermedad Cardiaca (Heart_Disease)')
plt.xlabel('Enfermedad Cardiaca')
plt.ylabel('Cantidad')
plt.xticks([0, 1], ['No (0)', 'Sí (1)'])  # Ajustar etiquetas si es necesario
plt.show()

# 4. Información adicional
print("\nTotal de casos:")
print(df['Heart_Disease'].value_counts())
print(f"\nPorcentaje de casos positivos: {distribucion[1]:.2f}%")

**Salida esperada:**  
- Un gráfico de barras mostrando ~22% "Yes" y ~78% "No".  
**Acción:** Si el desbalance es crítico, usaremos técnicas como **SMOTE** o **class_weight** en los modelos.

---

#### **2. Codificación de Variables Categóricas**  
**Problema:**  
- Variables como `General_Health` (5 categorías) o `Age_Category` (13 categorías) son ordinales/nominales.  
**Solución:**  
- **One-Hot Encoding** para categóricas nominales (ej: `Sex`).  
- **Label Encoding** para ordinales (ej: `General_Health` si tiene jerarquía como "Poor" < "Fair" < "Good").  

```python
from sklearn.preprocessing import LabelEncoder

# Ejemplo de Label Encoding para General_Health (si es ordinal)
health_rank = {"Poor": 0, "Fair": 1, "Good": 2, "Very Good": 3, "Excellent": 4}
df["General_Health_Encoded"] = df["General_Health"].map(health_rank)

# One-Hot Encoding para categóricas nominales (ej: Sex, Diabetes)
nominal_cols = ["Sex", "Diabetes", "Smoking_History"]
df_encoded = pd.get_dummies(df, columns=nominal_cols, drop_first=True)

# Eliminar columnas originales no codificadas
df_processed = df_encoded.drop(["General_Health", "Checkup"], axis=1)  # Ejemplo: eliminar columnas redundantes
```

---

#### **3. Normalización de Variables Numéricas**  
**Problema:**  
- Variables como `Height_(cm)` y `Alcohol_Consumption` tienen escalas muy diferentes.  
**Solución:**  
- **StandardScaler** para algoritmos sensibles a la escala (SVM, Regresión Logística).  

```python
from sklearn.preprocessing import StandardScaler

# Lista de columnas numéricas
numeric_cols = ["Height_(cm)", "Weight_(kg)", "BMI", "Alcohol_Consumption", "Fruit_Consumption"]

# Escalado
scaler = StandardScaler()
df_processed[numeric_cols] = scaler.fit_transform(df_processed[numeric_cols])
```

---

#### **4. División Train/Test Estratificada**  
**Por qué estratificar:**  
- Para mantener la proporción de la variable objetivo en ambos conjuntos.  

```python
from sklearn.model_selection import train_test_split

# Separar features y target
X = df_processed.drop("Heart_Disease", axis=1)
y = df_processed["Heart_Disease"]

# Split estratificado
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print(f"Train: {X_train.shape}, Test: {X_test.shape}")
```

---

#### **5. Análisis de Correlación (Opcional)**  
```python
# Matriz de correlación para variables numéricas
corr_matrix = df_processed[numeric_cols + ["General_Health_Encoded"]].corr()
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm")
plt.title("Matriz de Correlación")
plt.show()
```
**Qué buscar:**  
- Correlaciones altas (ej: `Weight` y `BMI`). Si hay redundancia, eliminar una.  

---

### **Próximos Pasos (Después de Ejecutar Este Código)**  
1. **Selección de Variables:**  
   - Usar `SelectKBest` o importancia con Random Forest para elegir las mejores features.  
2. **Entrenar Modelos Básicos:**  
   - Regresión Logística (baseline).  
   - Random Forest (para comparar).  
   - Ajustar hiperparámetros para evitar overfitting (max_depth, min_samples_leaf).  

**Ejemplo de modelo inicial (para siguiente paso):**  
```python
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

model = LogisticRegression(class_weight="balanced")  # Maneja desbalance
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))
```

---

### **Resumen de Acciones Clave**  
| Paso | Tarea | Justificación |  
|------|-------|---------------|  
| 1 | Analizar desbalance en `Heart_Disease` | Determinar si se necesita SMOTE o class_weight |  
| 2 | Codificar categóricas | One-Hot para nominales, Label para ordinales |  
| 3 | Escalar numéricas | Evitar sesgo en modelos sensibles a escala |  
| 4 | Split estratificado | Mantener proporción de clases en train/test |  

**Nota:** Comparte los resultados de la codificación y correlación para ajustar la selección de features.